<a href="https://colab.research.google.com/github/BKover99/Pseudovisium/blob/main/QC_mouse_brain_merscope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Pseudovisium -q


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.8/277.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.0 MB/

In [ ]:
import requests
import subprocess
import shutil
import os
from bs4 import BeautifulSoup
from Pseudovisium.pseudovisium_generate import generate_pv
from Pseudovisium.pseudovisium_qc import generate_qc_report



/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spaghetti/network.py:42: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)


In [ ]:
# URL of the webpage
url = "https://download.brainimagelibrary.org/aa/79/aa79b8ba5b3add56/638850/"

# Send a GET request to fetch the webpage content
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags which contain the folder names
    folder_links = soup.find_all('a')
    filenames=[]
    for i,folder_link in enumerate(folder_links[1:]):
        folder_name = folder_link.text.strip('/')
        if folder_name != "..":
            # Construct the full URL of the CSV file
            csv_url = url + folder_name + "/merfish_output/cellpose_cyto2_nuclei/cellpose-detected_transcripts.csv"
            filename= folder_name + "_cellpose-detected_transcripts.csv"
            filenames.append(filename)
            # Define the command to download using wget
            wget_command = f"wget -O {filename} {csv_url}"
            # Execute the wget command
            subprocess.run(wget_command, shell=True)
            print(f"Downloaded {folder_name}_cellpose-detected_transcripts.csv")

        print(f"Downloaded {folder_name}_cellpose-detected_transcripts.csv")
        csv_file = "/content/"+folder_name+"_cellpose-detected_transcripts.csv"
        output_path = "/content"
        hexagon_size = 50

        pseudovisium_path = generate_pv(csv_file=csv_file,
                                        hexagon_size=hexagon_size,
                                        output_path=output_path,
                                          batch_size=8000000,
                                          max_workers=10,
                                          x_colname = "global_x",
                                          y_colname = "global_y",
                                          feature_colname = "gene",
                                          cell_id_colname = "cell_id",
                                          technology="merscope",
                                          project_name="section"+str(i))

        os.remove(csv_file)
else:
    print("Failed to retrieve webpage. Status code:", response.status_code)

Downloaded 1199650929_cellpose-detected_transcripts.csv
Downloaded 1199650929_cellpose-detected_transcripts.csv
You are using version:  ['Pseudovisium==0.0.40']
Date:  2024-06-10
Technology not recognized. Going forward with set column names.
Quality filter is set to False
Quality counting per hexagon is set to False
Quality counting per probe is set to False
Created temporary directory /tmp/tmp_hexaz2lxw156
Now creating batches
Created batch 1
Created batch 2
Created batch 3
Created batch 4
Finished preprocessing. Total batches created: 4
Processing batches using 8 processes


Processing batches: 100%|██████████| 4/4 [01:04<00:00, 16.21s/batch]


Creating pseudovisium folder in output path:/content/pseudovisium/section0
Creating scalefactors_json.json file in spatial folder.
Creating barcodes.tsv.gz file in spatial folder.
Creating tissue_positions_list.csv file in spatial folder.
Creating pv_cell_hex.csv file in spatial folder.
No quality information provided. Skipping quality information files.
No quality information provided. Skipping quality information files.
Creating features.tsv.gz file in spatial folder.
Putting together the matrix.mtx file
Total matrix count: 29762163.0
Number of unique hexagons: 2707
Creating matrix.mtx.gz file in spatial folder.


/usr/local/lib/python3.10/dist-packages/Pseudovisium/pseudovisium_generate.py:786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hexagon_cell_counts["hexagon_id"] = hexagon_cell_counts["hexagon_id"] + 1


Putting together the filtered_feature_bc_matrix.h5 file
No image file provided. Drawing tissue positions on a white background.
Creating arguments.json file in output path.
Time taken: 292.244038105011 seconds
Searching for temporary batch files in: /tmp
No remaining temporary batch files found.
Downloaded 1199650932_cellpose-detected_transcripts.csv
Downloaded 1199650932_cellpose-detected_transcripts.csv
You are using version:  ['Pseudovisium==0.0.40']
Date:  2024-06-10
Technology not recognized. Going forward with set column names.
Quality filter is set to False
Quality counting per hexagon is set to False
Quality counting per probe is set to False
Created temporary directory /tmp/tmp_hexa4e40oc5w
Now creating batches
Created batch 1
Created batch 2
Created batch 3
Created batch 4
Finished preprocessing. Total batches created: 4
Processing batches using 8 processes


...

Putting together the filtered_feature_bc_matrix.h5 file
No image file provided. Drawing tissue positions on a white background.
Creating arguments.json file in output path.
Time taken: 677.285224199295 seconds
Searching for temporary batch files in: /tmp
No remaining temporary batch files found.
Downloaded 1199651106_cellpose-detected_transcripts.csv
Downloaded 1199651106_cellpose-detected_transcripts.csv
You are using version:  ['Pseudovisium==0.0.40']
Date:  2024-06-11
Technology not recognized. Going forward with set column names.
Quality filter is set to False
Quality counting per hexagon is set to False
Quality counting per probe is set to False
Created temporary directory /tmp/tmp_hexad7cr_m8k
Now creating batches
Created batch 1
Created batch 2
Created batch 3
Created batch 4
Created batch 5
Created batch 6
Created batch 7
Created batch 8
Finished preprocessing. Total batches created: 8
Processing batches using 8 processes


Processing batches: 100%|██████████| 8/8 [01:52<00:00, 14.02s/batch]


Creating pseudovisium folder in output path:/content/pseudovisium/section51
Using already existing folder: /content/pseudovisium/
Creating scalefactors_json.json file in spatial folder.
Creating barcodes.tsv.gz file in spatial folder.
Creating tissue_positions_list.csv file in spatial folder.
Creating pv_cell_hex.csv file in spatial folder.
No quality information provided. Skipping quality information files.
No quality information provided. Skipping quality information files.
Creating features.tsv.gz file in spatial folder.
Putting together the matrix.mtx file
Total matrix count: 58116822.0
Number of unique hexagons: 3579
Creating matrix.mtx.gz file in spatial folder.


/usr/local/lib/python3.10/dist-packages/Pseudovisium/pseudovisium_generate.py:786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hexagon_cell_counts["hexagon_id"] = hexagon_cell_counts["hexagon_id"] + 1


Putting together the filtered_feature_bc_matrix.h5 file
No image file provided. Drawing tissue positions on a white background.
Creating arguments.json file in output path.
Time taken: 537.8067779541016 seconds
Searching for temporary batch files in: /tmp
No remaining temporary batch files found.
Downloaded 1199651109_cellpose-detected_transcripts.csv
Downloaded 1199651109_cellpose-detected_transcripts.csv
You are using version:  ['Pseudovisium==0.0.40']
Date:  2024-06-11
Technology not recognized. Going forward with set column names.
Quality filter is set to False
Quality counting per hexagon is set to False
Quality counting per probe is set to False
Created temporary directory /tmp/tmp_hexa7k3c2w2l
Now creating batches
Created batch 1
Created batch 2
Created batch 3
Created batch 4


In [ ]:
folders= ["/content/pseudovisium/section"+str(i)  for i in range(60)]
folders

In [ ]:
output_folder="/content/"
gene_names=["Kitl","Fstl5","Slit2"]
generate_qc_report(folders,
                   output_folder,
                   gene_names,
                   include_morans_i=False,
                   max_workers=10,
                   normalisation=True,
                   save_plots=True,
                   squidpy=True)

In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Create the ZIP archive
!zip -r pseudovisium.zip /content/pseudovisium

# Copy the ZIP file to your Google Drive
!cp pseudovisium.zip /content/drive/MyDrive/

print("pseudovisium.zip has been saved to your Google Drive.")

Mounted at /content/drive
  adding: content/pseudovisium/ (stored 0%)
  adding: content/pseudovisium/section45/ (stored 0%)
  adding: content/pseudovisium/section45/spatial/ (stored 0%)
  adding: content/pseudovisium/section45/spatial/tissue_positions_list.csv (deflated 79%)
  adding: content/pseudovisium/section45/spatial/tissue_lowres_image.png (deflated 86%)
  adding: content/pseudovisium/section45/spatial/pv_cell_hex.csv (deflated 80%)
  adding: content/pseudovisium/section45/spatial/scalefactors_json.json (deflated 35%)
  adding: content/pseudovisium/section45/spatial/tissue_hires_image.png (deflated 94%)
  adding: content/pseudovisium/section45/matrix.mtx (deflated 87%)
  adding: content/pseudovisium/section45/barcodes.tsv (deflated 81%)
  adding: content/pseudovisium/section45/features.tsv.gz (deflated 3%)
  adding: content/pseudovisium/section45/barcodes.tsv.gz (deflated 70%)
  adding: content/pseudovisium/section45/features.tsv (deflated 81%)
  adding: content/pseudovisium/sec